In [ ]:
# Credits: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

def plt_dynamic(x, vy, ty, ax, colors=['b']):
    ax.plot(x, vy, 'b', label="Validation Loss")
    ax.plot(x, ty, 'r', label="Train Loss")
    plt.legend()
    plt.grid()
    fig.canvas.draw()

In [ ]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

<h2> 1. CNN + Dropout + AdadeltaOptimizer </h2>

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,\
              optimizer=keras.optimizers.Adadelta(),\
              metrics=['accuracy'])
history=model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') 
ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))
vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
58368/60000 [============================>.] - ETA: 4s - loss: 0.2633 - acc: 0.919560000/60000 [==============================] - 163s 3ms/step - loss: 0.2595 - acc: 0.9206 - val_loss: 0.0728 - val_acc: 0.9760
Epoch 2/12
56704/60000 [===========================>..] - ETA: 8s - loss: 0.0885 - acc: 0.973160000/60000 [==============================] - 163s 3ms/step - loss: 0.0875 - acc: 0.9734 - val_loss: 0.0412 - val_acc: 0.9855
Epoch 3/12
56064/60000 [===========================>..] - ETA: 10s - loss: 0.0653 - acc: 0.980360000/60000 [==============================] - 163s 3ms/step - loss: 0.0654 - acc: 0.9804 - val_loss: 0.0335 - val_acc: 0.9884
Epoch 4/12
55808/60000 [==========================>...] - ETA: 11s - loss: 0.0541 - acc: 0.983760000/60000 [==============================] - 172s 3ms/step - loss: 0.0543 - acc: 0.9838 - val_loss: 0.0311 - val_acc

<h2> 2. CNN + Batch Normalization +Dropout + AdamOptimizer </h2>

In [ ]:
model2 = Sequential()
model2.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=input_shape,\
                 kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model2.add(BatchNormalization())
model2.add(Conv2D(64, (5, 5), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(5, 5)))
model2.add(Dropout(0.25))

model2.add(Conv2D(72, (5, 5), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model2.add(BatchNormalization())
model2.add(Conv2D(84, (5, 5), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(5, 5)))
model2.add(Dropout(0.25))

model2.add(Conv2D(97, (5, 5), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(5, 5)))
model2.add(Dropout(0.25))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.5))

model2.add(Dense(num_classes, activation='softmax'))

model2.compile(loss=keras.losses.categorical_crossentropy,\
              optimizer=keras.optimizers.Adam(),\
              metrics=['accuracy'])
history=model2.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
score = model2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') 
ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))
vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

<h2> 2. CNN + Batch Normalization +Dropout+ Padding + stride + AdamOptimizer </h2>

In [ ]:
model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(7, 7),activation='relu',input_shape=input_shape,\
                 kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))
model3.add(Conv2D(64, (7, 7), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))
model3.add(Conv2D(72, (7, 7), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))
model3.add(MaxPooling2D(pool_size=(7,7),strides=(3,3), padding='valid'))
model3.add(Dropout(0.25))

model3.add(Conv2D(84, (7, 7), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))
model3.add(Conv2D(92, (7, 7), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))
model3.add(Conv2D(102, (7, 7), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))
model3.add(MaxPooling2D(pool_size=(7,7),strides=(3,3), padding='valid'))
model3.add(Dropout(0.25))

model3.add(Conv2D(108, (7, 7), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))
model3.add(Conv2D(114, (7, 7), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))
model3.add(Conv2D(122, (7, 7), activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))
model3.add(MaxPooling2D(pool_size=(7,7),strides=(3,3), padding='valid'))
model3.add(Dropout(0.25))
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.5))

model3.add(Dense(num_classes, activation='softmax'))

model3.compile(loss=keras.losses.categorical_crossentropy,\
              optimizer=keras.optimizers.Adam(),\
              metrics=['accuracy'])
history=model3.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
score = model3.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') 
ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))
vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)